In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import os
import torch
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
import numpy as np

In [2]:
# Load the training data
train_data = pd.read_csv('../../2nd-Comp-Data/train.csv')
test_data = pd.read_csv('../../2nd-Comp-Data/test.csv')

used = []

# Extract features and target variable
X = train_data.drop('price_doc', axis=1)
y = train_data['price_doc']
X_test = test_data.drop(['row ID'], axis=1)

In [3]:
# X = X.drop('sub_area', axis=1)
# X_test = X_test.drop('sub_area', axis=1)
# used.append('Removed sub_area')

In [4]:
# X = pd.get_dummies(X)
# X_test = pd.get_dummies(X_test) 
# used.append('OneHot Encoding')

In [5]:
from sklearn.calibration import LabelEncoder

categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
print("Train: Categorical columns:", categorical_columns)

label_encoder = LabelEncoder()

for column in categorical_columns:
    X[column] = label_encoder.fit_transform(X[column])

categorical_columns_test = X_test.select_dtypes(include=['object']).columns.tolist()
print("Test: Categorical columns:", categorical_columns_test)

label_encoder = LabelEncoder()

for column in categorical_columns_test:
    X_test[column] = label_encoder.fit_transform(X_test[column])

used.append('Label Encoding')

Train: Categorical columns: ['product_type', 'sub_area', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology']
Test: Categorical columns: ['product_type', 'sub_area', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology']


In [6]:
# # drop all columns in X_train with dtypes object
# for col in X.columns:
#     if X[col].dtype == 'object':
#         X.drop(col, axis=1, inplace=True)

# # drop all columns in X_test with dtypes object
# for col in X_test.columns:
#     if X_test[col].dtype == 'object':
#         X_test.drop(col, axis=1, inplace=True)

# used.append("Removed Object Dtypes")

In [7]:
X = X.astype('float32')
X_test = X_test.astype('float32')
used.append("Converted All Columns To float32")

In [8]:
X = np.log1p(X)
X_test = np.log1p(X_test)
used.append('log Normalization')

In [9]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
used.append("StandardScaler")

<h1>With Keras<h1>

In [11]:
features_used = X_train.shape[1]

# optimizerUsing = SGD(learning_rate=0.01, momentum=0.9)

# optimizerUsing = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
optimizerUsing = Adam(lr=0.001)#, epsilon=1e-07)

# Build the neural network
os.environ["KERAS_BACKEND"] = "torch"
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))  # You can use either dropout or early stopping
model.add(Dense(68, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(40, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))  # Output layer with linear activation for regression

# Compile the model
model.compile(loss='mean_squared_error', optimizer=optimizerUsing)

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=1800, validation_data=(X_val, y_val), callbacks=[early_stopping])

used.append("Keras With Torch Backend")
# used.append("1 Hidden Layer: Hidden Layer 1 with 140 neurons relu activation, Output linear")
# used.append("2 Hidden Layers: Hidden Layer 1 with 100 neurons relu activation, Hidden Layer 2 with 50 neurons relu activation, Dropout 0.3, Output linear")
used.append("3 Hidden Layers: Hidden Layer 1 with 128 neurons relu activation L2 regularization, Hidden Layer 2 with 64 neurons relu activation L2 regularization, Hidden Layer 3 with 40 neurons relu activation L2 regularization")
used.append("Output linear")
used.append("Optimizer: Adam(lr=0.01)")
used.append("Loss Calculation: Mean Squared Error")
used.append("EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)")
used.append("batch_size=1800, epochs=100, early_stopping, dropout=0.3 (used in hidden layers only)")
# used.append("batch_size=1800, epochs=100, early_stopping")

# Make predictions
predictions = model.predict(X_test)

Epoch 1/100
71/71 [==============================] - 1s 7ms/step - loss: 694002156306432.0000 - val_loss: 694744246124544.0000
Epoch 2/100
71/71 [==============================] - 0s 5ms/step - loss: 692944721936384.0000 - val_loss: 691638582116352.0000
Epoch 3/100
71/71 [==============================] - 0s 5ms/step - loss: 683798589079552.0000 - val_loss: 673166129102848.0000
Epoch 4/100
71/71 [==============================] - 0s 5ms/step - loss: 648513151041536.0000 - val_loss: 616089369182208.0000
Epoch 5/100
71/71 [==============================] - 0s 5ms/step - loss: 563956653490176.0000 - val_loss: 501316535189504.0000
Epoch 6/100
71/71 [==============================] - 0s 5ms/step - loss: 427536781672448.0000 - val_loss: 348296681881600.0000
Epoch 7/100
71/71 [==============================] - 0s 5ms/step - loss: 288458459316224.0000 - val_loss: 230829779320832.0000
Epoch 8/100
71/71 [==============================] - 0s 5ms/step - loss: 215472502996992.0000 - val_loss: 19038

<h1>With Skilearn<h1>

In [12]:
# features_used = X_train.shape[1]

# # Build the neural network using MLPRegressor from scikit-learn
# model = MLPRegressor(hidden_layer_sizes=(140), activation='relu', solver='adam', max_iter=100,
#                      batch_size=1800, early_stopping=True, random_state=42)

# # Train the model
# model.fit(X_train, y_train)

# used.append("MLPRegressor")
# used.append("1 Hidden Layer: Hidden Layer 1 with 140 neurons relu activation, Output linear")
# # used.append("2 Hidden Layers: Hidden Layer 1 with 128 neurons relu activation, \nHidden Layer 2 with 64 neurons relu activation, Output linear")
# used.append("Adam Optimizer")
# # used.append("Loss Calculation: Mean Squared Error")
# used.append("EarlyStopping")
# used.append("batch_size=1800, epochs=100, early_stopping, dropout=0.3 (used in hidden layers only)")

# # Make predictions
# predictions = model.predict(X_test)

In [13]:
# Create a DataFrame with the results
submission_df = pd.DataFrame({'row ID': test_data['row ID'], 'price_doc': predictions.flatten()})

# Save the results to a CSV file
submission_df.to_csv('Day7.8.csv', index=False)

In [14]:
print("Features Used = " + str(features_used) + "\n")
for i in used:
    print(i)

Features Used = 271

Label Encoding
Converted All Columns To float32
log Normalization
StandardScaler
Keras With Torch Backend
3 Hidden Layers: Hidden Layer 1 with 128 neurons relu activation L2 regularization, Hidden Layer 2 with 64 neurons relu activation L2 regularization, Hidden Layer 3 with 40 neurons relu activation L2 regularization
Output linear
Optimizer: Adam(lr=0.01)
Loss Calculation: Mean Squared Error
EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
batch_size=1800, epochs=100, early_stopping, dropout=0.3 (used in hidden layers only)
